Testing CNN method from https://wandb.ai/mostafaibrahim17/ml-articles/reports/An-Introduction-to-Audio-Classification-with-Keras--Vmlldzo0MDQzNDUy#:~:text=%EF%BB%BFKeras%20is%20a%20go,both%20beginners%20and%20advanced%20users.

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
import matplotlib.pyplot as plt

In [2]:
#import wandb
#wandb.init(project='audio_classification_using_W&B', 
#           entity='enter your entity name here', 
#           config={"batch_size": 32, "epochs": 100})

Depending on what data we are using we may not need this

In [31]:
def get_data(audio_path, meta_file, label_name):
    songs_matrix = []
    songs_label = []
    audio_list = os.listdir(audio_path)
    df = pd.read_csv(meta_file)
    for name in audio_list:
        #name = audio_list[0]
        number = name.replace('_', '.').split('.')[0]

        song, sr = librosa.load(os.path.join(audio_path, name))
        mfccs = librosa.feature.mfcc(y = song, n_mfcc = 20)
        mfcc = np.mean(mfccs, axis=1) # average over columns, shape(n_mfcc, )

        songs_matrix.append(mfcc)
        songs_label.append(df[label_name].loc[df.cat_num == float(number)].item())
    return np.array(songs_matrix), np.array(songs_label)

In [32]:
# load all audio in audio_path
# convert to matrix, save in np array, 
# load the corresponding label_name from lable_filename, save in another np array

audio_path = 'Small_audio_sample'
meta_file = 'MLNS_Insects_Fams.csv'
label_name = 'fam_or_subfam'

songs_matrix, songs_label = get_data(audio_path, meta_file, label_name)

In [81]:
print(songs_label)

['Gryllotalpidae' 'Trigonidiinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Conocephalinae']


In [ ]:
# convert insec fam into numbers. use le.inverse_transform to go from number to name
le = LabelEncoder()
songs_label_num = le.fit_transform(songs_label)
songs_label_onehot = to_categorical(songs_label_num) # why do we do this??

In [131]:
## if ever need to convert onehot matrix back to string label
#label_num = np.sum(songs_label_onehot*np.arange(0,songs_label_onehot.shape[1]), axis=1)
#print(le.inverse_transform(label_num.astype(int)))

['Gryllotalpidae' 'Trigonidiinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Conocephalinae']


In [44]:
X_train, X_test, y_train, y_test = train_test_split(songs_matrix, songs_label_onehot, 
                                                    test_size = .2, random_state = 404,
                                                    )

In [110]:
y_test = y_test.astype(int)
print(list(y_test[:,1] + 2*y_test[:,2]+3*y_test[:,3]))
print(le.inverse_transform(y_test[:,1] + 2*y_test[:,2]+3*y_test[:,3]))

[2, 1, 2, 2, 2, 2]
['Phaneropterinae' 'Gryllotalpidae' 'Phaneropterinae' 'Phaneropterinae'
 'Phaneropterinae' 'Phaneropterinae']


"A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor."
https://keras.io/guides/sequential_model/

In [75]:
input_shape = (X_train.shape[1], 1) # n_featuer-by-1 vector
model = Sequential() 
model.add(Conv1D(64, 3, padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv1D(128, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(le.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [78]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [80]:
predicted_vector = model.predict(X_test)
predicted_class_index = np.argmax(predicted_vector, axis=-1)
le.inverse_transform(predicted_class_index)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array(['Conocephalinae', 'Conocephalinae', 'Conocephalinae',
       'Conocephalinae', 'Conocephalinae', 'Conocephalinae'], dtype='<U15')

In [77]:
x = np.random.rand(20,50)
print(x.shape)
y = Conv1D(32, 5, activation='relu')(x)
print(y.shape)

(20, 50)


ValueError: Input 0 of layer "conv1d_14" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (20, 50)